# Eat Safe, Love

## Notebook Set Up

In [78]:
# Import dependencies
from pymongo import MongoClient
import json
from pprint import pprint
import pandas as pd
import re

In [79]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [80]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [81]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [82]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [83]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene" : 20}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63f30d75495a581d25b65d86'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta

In [84]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.json_normalize(establishments.find(query))
# Display the number of rows in the DataFrame
print(len(hygiene_df))
# Display the first 10 rows of the DataFrame
hygiene_df.head(20)

40


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,63f30d75495a581d25b66108,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,1.278721,51.857536,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,63f30d75495a581d25b66412,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,1.15927,51.789429,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,63f30d75495a581d25b66601,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,0.10202,50.770885,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,63f30d75495a581d25b6660e,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,0.101446,50.770724,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,63f30d75495a581d25b66fae,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,0.736349,51.541448,None,0001-01-01T00:00:00,0,None,0,0,0,0
5,63f30d75495a581d25b671cf,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,0.7121671,51.5350065,None,0001-01-01T00:00:00,0,None,0,0,0,0
6,63f30d75495a581d25b686fc,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,0.602364,51.591515,None,0001-01-01T00:00:00,0,None,0,0,0,0
7,63f30d75495a581d25b68b3e,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,0.146449998021126,51.1859474182129,None,0001-01-01T00:00:00,0,None,0,0,0,0
8,63f30d75495a581d25b6934f,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,0.462307,51.57005,None,0001-01-01T00:00:00,0,None,0,0,0,0
9,63f30d75495a581d25b693bf,267034,0,11/00102/MIXED,Longhouse,Restaurant/Cafe/Canteen,1,17 London Road,Riverhead,KENT,...,0.169685,51.284795,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [85]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
regex = re.compile(r".*London.*", re.IGNORECASE)
query = {"LocalAuthorityName": regex, "RatingValue": {"$gte": "4"}}
# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

34
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63f30d75495a581d25b677a2'),
 'geocode': {'latitude': Decimal128('51.369321'),
             'longitude': Decimal128('0.508551')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/estab

In [86]:
# Convert the result to a Pandas DataFrame
London_Rating_4_more = pd.json_normalize(establishments.find(query))
# Display the number of rows in the DataFrame
print(len(London_Rating_4_more))
# Display the first 10 rows of the DataFrame
London_Rating_4_more.head(10)

33


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,63f30d75495a581d25b67ac8,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,0.249255999922752,51.1236228942871,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,63f30d75495a581d25b68614,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,0.561954,51.543831,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,63f30d75495a581d25b69411,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,0.346336990594864,51.4640655517578,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,63f30d75495a581d25b69414,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,0.346325,51.464078,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,63f30d75495a581d25b69b1e,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,0.287555,51.504071,None,0001-01-01T00:00:00,0,None,0,0,0,0
5,63f30d76495a581d25b6bc28,1252616,0,PI/000035653,Froottree,Other catering premises,7841,Tereza Joanne King George V Lock,Woolwich Manor Way London,Charlton,...,0.0693250000476837,51.5011100769043,None,0001-01-01T00:00:00,0,None,0,0,0,0
6,63f30d76495a581d25b6bc2c,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,0.069286,51.501121,None,0001-01-01T00:00:00,0,None,0,0,0,0
7,63f30d76495a581d25b6bff1,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,0.048006,51.503733,None,0001-01-01T00:00:00,0,None,0,0,0,0
8,63f30d76495a581d25b6c00d,294606,0,PI/000016185,WH Smith,Retailers - other,4613,London City Airport,Silvertown London,,...,0.0479655750095844,51.5041122436523,None,0001-01-01T00:00:00,0,None,0,0,0,0
9,63f30d76495a581d25b6c00e,293772,0,PI/000002585,The Nuance Group (UK) Limited,Retailers - other,4613,Duty Free Shop Passenger Terminal,London City AirportRoyal DocksLondon,,...,0.047966,51.504112,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [114]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
query = { "BusinessName": "Penang Flavours" }
result = establishments.find(query)
# save penang latitude and longitude
for doc in result:
    penang_lat = doc["geocode"]["latitude"]
    penang_long = doc["geocode"]["longitude"]
    
degree_search = 0.01

# Filter the MongoDB collection for restaurants with RatingValue of 5

# Filter the MongoDB collection for restaurants with RatingValue of 5
filter_query = {
    'RatingValue': '5',
    'geocode.latitude': {'$gt': {'$toDecimal':penang_lat - (0.01)}, '$lt': {'$toDecimal':penang_lat + (0.01)}},
    'geocode.longitude': {'$gt': {'$toDecimal':penang_long - (0.01)}, '$lt': {'$toDecimal':penang_long + (0.01)}
}}
print(establishments.count_documents(filter_query))
# # sort =  

# Print the results


TypeError: unsupported operand type(s) for -: 'Decimal128' and 'float'

In [108]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
